In [1]:
import gymnasium as gym
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, Flatten, Dense
from keras.optimizers import Adam
import cv2
import random
from collections import deque
import time
import os

C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [16]:
# Preprocessing function
def preprocess_frame(frame):
    """Convert RGB data to grayscale. Keep original dimensions (210x160)"""
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    normalized_frame = gray_frame / 255.0  # Normalize pixel values
    return normalized_frame

In [3]:
def build_model(action_size):
    """CNN architecture"""
    model = Sequential()
    model.add(Conv2D(32, (8, 8), strides=(4, 4), activation='relu', input_shape=(210, 160, 1)))
    model.add(Conv2D(64, (4, 4), strides=(2, 2), activation='relu'))
    model.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(action_size, activation='linear'))
    
    model.compile(loss='mse', optimizer=Adam(learning_rate=0.00025))
    return model

In [ ]:
# System for progressively saving weights as the CNN trains
def create_memory(capacity=100000):
    """Create a memory buffer for experience replay"""
    return deque(maxlen=capacity)

def add_to_memory(memory, state, action, reward, next_state, done):
    """Add experience to memory"""
    memory.append((state, action, reward, next_state, done))

def sample_from_memory(memory, batch_size):
    """Sample random batch from memory"""
    return random.sample(memory, batch_size

In [ ]:
# Epsilon-greedy policy -- balances exploration and exploitation
def epsilon_greedy_action(model, state, epsilon, action_size):
    """Choose action using epsilon-greedy policy"""
    if np.random.random() <= epsilon:
        return random.randrange(action_size)  # Explore: choose random action
    else:
        # Exploit: choose best action
        q_values = model.predict(np.expand_dims(state, axis=0), verbose=0)[0]
        return np.argmax(q_values)  # Choose action with highest Q-value

In [ ]:
# Training function
def train_dqn(episodes=10000, 
              max_steps=50000, 
              batch_size=32, 
              gamma=0.99, 
              epsilon_start=1.0, 
              epsilon_end=0.1, 
              epsilon_decay=0.995,
              update_target_freq=10000,
              memory_capacity=100000,
              save_freq=100):
    """Train a DQN model on Frogger"""
    # Create environment
    env = gym.make('ALE/Frogger-v5')
    action_size = env.action_space.n
    
    # Create main and target models
    main_model = build_model(action_size)
    target_model = build_model(action_size)
    target_model.set_weights(main_model.get_weights())  # Initialize target with same weights
    
    # Create memory for experience replay
    memory = create_memory(capacity=memory_capacity)
    
    # Training metrics
    total_steps = 0
    epsilon = epsilon_start
    
    # Model saving directory
    save_dir = "frogger_model"
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    # Training loop
    for episode in range(1, episodes + 1):
        # Reset environment and get initial state
        frame, info = env.reset()
        state = preprocess_frame(frame)
        state = np.expand_dims(state, axis=-1)  # Add channel dimension: (120, 160, 1)
        
        episode_reward = 0
        
        for step in range(max_steps):
            # Choose action
            action = epsilon_greedy_action(main_model, state, epsilon, action_size)
            
            # Take action
            next_frame, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated
            
            # Prepare next state
            next_state = preprocess_frame(next_frame)
            next_state = np.expand_dims(next_state, axis=-1)  # Add channel dimension
            
            # Store experience in memory
            add_to_memory(memory, state, action, reward, next_state, done)
            
            # Move to next state
            state = next_state
            episode_reward += reward
            total_steps += 1
            
            # Train with experience replay if memory has enough samples
            if len(memory) > batch_size:
                # Sample batch from memory
                minibatch = sample_from_memory(memory, batch_size)
                
                # Prepare batch for training
                states = np.array([experience[0] for experience in minibatch])
                actions = np.array([experience[1] for experience in minibatch])
                rewards = np.array([experience[2] for experience in minibatch])
                next_states = np.array([experience[3] for experience in minibatch])
                dones = np.array([experience[4] for experience in minibatch])
                
                # Calculate target Q values
                target_q_values = main_model.predict(states, verbose=0)
                next_q_values = target_model.predict(next_states, verbose=0)
                
                for i in range(batch_size):
                    if dones[i]:
                        target_q_values[i, actions[i]] = rewards[i]
                    else:
                        target_q_values[i, actions[i]] = rewards[i] + gamma * np.max(next_q_values[i])
                
                # Train the model
                main_model.fit(states, target_q_values, epochs=1, verbose=0)
            
            # Update target network periodically
            if total_steps % update_target_freq == 0:
                target_model.set_weights(main_model.get_weights())
                print(f"Target network updated at step {total_steps}")
            
            if done:
                break
        
        # Decay epsilon
        if epsilon > epsilon_end:
            epsilon *= epsilon_decay
        
        # Print episode stats
        print(f"Episode: {episode}, Reward: {episode_reward}, Epsilon: {epsilon:.4f}, Steps: {step+1}")
        
        # Save model periodically
        if episode % save_freq == 0:
            main_model.save(f"{save_dir}/frogger_dqn_episode_{episode}.h5")
            print(f"Model saved at episode {episode}")
    
    # Save final model
    main_model.save(f"{save_dir}/frogger_dqn_final.h5")
    print("Training completed!")
    
    env.close()
    return main_model

In [ ]:
# Main execution
if __name__ == "__main__":
    # You can adjust these parameters as needed
    train_dqn(episodes=1000,
              max_steps=10000, 
              batch_size=32, 
              gamma=0.99, 
              epsilon_start=1.0, 
              epsilon_end=0.01, 
              epsilon_decay=0.995,
              update_target_freq=1000,
              memory_capacity=50000,
              save_freq=50)

Episode: 1, Reward: 10.0, Epsilon: 0.9950, Steps: 271
Episode: 2, Reward: 7.0, Epsilon: 0.9900, Steps: 232
Episode: 3, Reward: 8.0, Epsilon: 0.9851, Steps: 277
Target network updated at step 1000
Episode: 4, Reward: 9.0, Epsilon: 0.9801, Steps: 290
Episode: 5, Reward: 8.0, Epsilon: 0.9752, Steps: 258
Episode: 6, Reward: 10.0, Epsilon: 0.9704, Steps: 307
Episode: 7, Reward: 8.0, Epsilon: 0.9655, Steps: 248
Target network updated at step 2000
Episode: 8, Reward: 9.0, Epsilon: 0.9607, Steps: 306
Episode: 9, Reward: 7.0, Epsilon: 0.9559, Steps: 262
Episode: 10, Reward: 13.0, Epsilon: 0.9511, Steps: 281
Episode: 11, Reward: 11.0, Epsilon: 0.9464, Steps: 248
Target network updated at step 3000
Episode: 12, Reward: 9.0, Epsilon: 0.9416, Steps: 217
Episode: 13, Reward: 7.0, Epsilon: 0.9369, Steps: 273
Episode: 14, Reward: 9.0, Epsilon: 0.9322, Steps: 325
Target network updated at step 4000
Episode: 15, Reward: 8.0, Epsilon: 0.9276, Steps: 226
Episode: 16, Reward: 12.0, Epsilon: 0.9229, Steps: 

Episode: 50, Reward: 12.0, Epsilon: 0.7783, Steps: 279
Model saved at episode 50
Target network updated at step 14000
Episode: 51, Reward: 13.0, Epsilon: 0.7744, Steps: 270
Episode: 52, Reward: 8.0, Epsilon: 0.7705, Steps: 236
Episode: 53, Reward: 11.0, Epsilon: 0.7667, Steps: 285
Episode: 54, Reward: 10.0, Epsilon: 0.7629, Steps: 259
Target network updated at step 15000
Episode: 55, Reward: 8.0, Epsilon: 0.7590, Steps: 229
Episode: 56, Reward: 7.0, Epsilon: 0.7553, Steps: 249
Episode: 57, Reward: 14.0, Epsilon: 0.7515, Steps: 275
Episode: 58, Reward: 11.0, Epsilon: 0.7477, Steps: 245
Target network updated at step 16000
Episode: 59, Reward: 10.0, Epsilon: 0.7440, Steps: 320
Episode: 60, Reward: 9.0, Epsilon: 0.7403, Steps: 279
Episode: 61, Reward: 8.0, Epsilon: 0.7366, Steps: 210
Target network updated at step 17000
Episode: 62, Reward: 10.0, Epsilon: 0.7329, Steps: 308
Episode: 63, Reward: 11.0, Epsilon: 0.7292, Steps: 260
Episode: 64, Reward: 8.0, Epsilon: 0.7256, Steps: 252
Episode

Episode: 100, Reward: 8.0, Epsilon: 0.6058, Steps: 233
Model saved at episode 100
Episode: 101, Reward: 16.0, Epsilon: 0.6027, Steps: 287
Target network updated at step 28000
Episode: 102, Reward: 11.0, Epsilon: 0.5997, Steps: 318
Episode: 103, Reward: 12.0, Epsilon: 0.5967, Steps: 336
Episode: 104, Reward: 12.0, Epsilon: 0.5937, Steps: 257
Target network updated at step 29000
Episode: 105, Reward: 11.0, Epsilon: 0.5908, Steps: 305
Episode: 106, Reward: 15.0, Epsilon: 0.5878, Steps: 279
Episode: 107, Reward: 11.0, Epsilon: 0.5849, Steps: 306
Episode: 108, Reward: 14.0, Epsilon: 0.5820, Steps: 311
Target network updated at step 30000
Episode: 109, Reward: 14.0, Epsilon: 0.5790, Steps: 286
Episode: 110, Reward: 10.0, Epsilon: 0.5762, Steps: 229
Episode: 111, Reward: 13.0, Epsilon: 0.5733, Steps: 273
Target network updated at step 31000
Episode: 112, Reward: 18.0, Epsilon: 0.5704, Steps: 310
Episode: 113, Reward: 16.0, Epsilon: 0.5676, Steps: 345
Episode: 114, Reward: 15.0, Epsilon: 0.564

Episode: 150, Reward: 18.0, Epsilon: 0.4715, Steps: 321
Model saved at episode 150
Episode: 151, Reward: 19.0, Epsilon: 0.4691, Steps: 327
Target network updated at step 43000
Episode: 152, Reward: 14.0, Epsilon: 0.4668, Steps: 335
Episode: 153, Reward: 12.0, Epsilon: 0.4644, Steps: 327
Episode: 154, Reward: 16.0, Epsilon: 0.4621, Steps: 313
Target network updated at step 44000
Episode: 155, Reward: 18.0, Epsilon: 0.4598, Steps: 352
Episode: 156, Reward: 14.0, Epsilon: 0.4575, Steps: 309
Episode: 157, Reward: 9.0, Epsilon: 0.4552, Steps: 302
Target network updated at step 45000
Episode: 158, Reward: 11.0, Epsilon: 0.4529, Steps: 307
Episode: 159, Reward: 11.0, Epsilon: 0.4507, Steps: 236
Episode: 160, Reward: 11.0, Epsilon: 0.4484, Steps: 300
Target network updated at step 46000
Episode: 161, Reward: 18.0, Epsilon: 0.4462, Steps: 389
Episode: 162, Reward: 11.0, Epsilon: 0.4440, Steps: 265
Episode: 163, Reward: 12.0, Epsilon: 0.4417, Steps: 319
Episode: 164, Reward: 10.0, Epsilon: 0.439

Episode: 200, Reward: 15.0, Epsilon: 0.3670, Steps: 282
Model saved at episode 200
Episode: 201, Reward: 16.0, Epsilon: 0.3651, Steps: 303
Target network updated at step 58000
Episode: 202, Reward: 11.0, Epsilon: 0.3633, Steps: 321
Episode: 203, Reward: 17.0, Epsilon: 0.3615, Steps: 265
Episode: 204, Reward: 20.0, Epsilon: 0.3597, Steps: 319
Target network updated at step 59000
Episode: 205, Reward: 19.0, Epsilon: 0.3579, Steps: 305
Episode: 206, Reward: 19.0, Epsilon: 0.3561, Steps: 330
Episode: 207, Reward: 12.0, Epsilon: 0.3543, Steps: 232
Episode: 208, Reward: 12.0, Epsilon: 0.3525, Steps: 302
Target network updated at step 60000
Episode: 209, Reward: 17.0, Epsilon: 0.3508, Steps: 323
Episode: 210, Reward: 18.0, Epsilon: 0.3490, Steps: 297
Episode: 211, Reward: 10.0, Epsilon: 0.3473, Steps: 251
Target network updated at step 61000
Episode: 212, Reward: 9.0, Epsilon: 0.3455, Steps: 315
Episode: 213, Reward: 12.0, Epsilon: 0.3438, Steps: 270
Episode: 214, Reward: 18.0, Epsilon: 0.342

...

Episode: 300, Reward: 19.0, Epsilon: 0.2223, Steps: 380
Model saved at episode 300
Episode: 301, Reward: 12.0, Epsilon: 0.2212, Steps: 393
Episode: 302, Reward: 15.0, Epsilon: 0.2201, Steps: 384
Target network updated at step 89000
Episode: 303, Reward: 16.0, Epsilon: 0.2190, Steps: 299
Episode: 304, Reward: 15.0, Epsilon: 0.2179, Steps: 281
Episode: 305, Reward: 19.0, Epsilon: 0.2168, Steps: 396
Target network updated at step 90000
Episode: 306, Reward: 13.0, Epsilon: 0.2157, Steps: 260
Episode: 307, Reward: 13.0, Epsilon: 0.2146, Steps: 273
Episode: 308, Reward: 20.0, Epsilon: 0.2136, Steps: 420
Target network updated at step 91000
Episode: 309, Reward: 16.0, Epsilon: 0.2125, Steps: 344
Episode: 310, Reward: 14.0, Epsilon: 0.2114, Steps: 319
Episode: 311, Reward: 18.0, Epsilon: 0.2104, Steps: 361
Target network updated at step 92000
Episode: 312, Reward: 17.0, Epsilon: 0.2093, Steps: 387
Episode: 313, Reward: 12.0, Epsilon: 0.2083, Steps: 334
Episode: 314, Reward: 18.0, Epsilon: 0.20

Episode: 350, Reward: 17.0, Epsilon: 0.1730, Steps: 390
Model saved at episode 350
Episode: 351, Reward: 20.0, Epsilon: 0.1721, Steps: 341
Target network updated at step 105000
Episode: 352, Reward: 12.0, Epsilon: 0.1713, Steps: 399
Episode: 353, Reward: 11.0, Epsilon: 0.1704, Steps: 278
Episode: 354, Reward: 18.0, Epsilon: 0.1696, Steps: 396
Episode: 355, Reward: 16.0, Epsilon: 0.1687, Steps: 286
Target network updated at step 106000
Episode: 356, Reward: 11.0, Epsilon: 0.1679, Steps: 317
Episode: 357, Reward: 14.0, Epsilon: 0.1670, Steps: 313
Episode: 358, Reward: 19.0, Epsilon: 0.1662, Steps: 333
Target network updated at step 107000
Episode: 359, Reward: 19.0, Epsilon: 0.1654, Steps: 285
Episode: 360, Reward: 19.0, Epsilon: 0.1646, Steps: 258
Episode: 361, Reward: 17.0, Epsilon: 0.1637, Steps: 299
Target network updated at step 108000
Episode: 362, Reward: 17.0, Epsilon: 0.1629, Steps: 228
Episode: 363, Reward: 17.0, Epsilon: 0.1621, Steps: 344
Episode: 364, Reward: 13.0, Epsilon: 

Episode: 400, Reward: 13.0, Epsilon: 0.1347, Steps: 284
Model saved at episode 400
Episode: 401, Reward: 19.0, Epsilon: 0.1340, Steps: 281
Episode: 402, Reward: 19.0, Epsilon: 0.1333, Steps: 279
Target network updated at step 120000
Episode: 403, Reward: 10.0, Epsilon: 0.1326, Steps: 233
Episode: 404, Reward: 16.0, Epsilon: 0.1320, Steps: 295
Episode: 405, Reward: 16.0, Epsilon: 0.1313, Steps: 379
Target network updated at step 121000
Episode: 406, Reward: 10.0, Epsilon: 0.1307, Steps: 233
Episode: 407, Reward: 13.0, Epsilon: 0.1300, Steps: 394
Episode: 408, Reward: 22.0, Epsilon: 0.1294, Steps: 362
Target network updated at step 122000
Episode: 409, Reward: 13.0, Epsilon: 0.1287, Steps: 281
Episode: 410, Reward: 14.0, Epsilon: 0.1281, Steps: 424
Episode: 411, Reward: 19.0, Epsilon: 0.1274, Steps: 289
Target network updated at step 123000
Episode: 412, Reward: 23.0, Epsilon: 0.1268, Steps: 415
Episode: 413, Reward: 10.0, Epsilon: 0.1262, Steps: 272
Episode: 414, Reward: 15.0, Epsilon: 

Episode: 450, Reward: 14.0, Epsilon: 0.1048, Steps: 281
Model saved at episode 450
Target network updated at step 137000
Episode: 451, Reward: 17.0, Epsilon: 0.1043, Steps: 486
Episode: 452, Reward: 18.0, Epsilon: 0.1038, Steps: 252
Episode: 453, Reward: 20.0, Epsilon: 0.1032, Steps: 318
Target network updated at step 138000
Episode: 454, Reward: 21.0, Epsilon: 0.1027, Steps: 329
Episode: 455, Reward: 19.0, Epsilon: 0.1022, Steps: 447
Episode: 456, Reward: 14.0, Epsilon: 0.1017, Steps: 377
Target network updated at step 139000
Episode: 457, Reward: 20.0, Epsilon: 0.1012, Steps: 305
Episode: 458, Reward: 16.0, Epsilon: 0.1007, Steps: 496
Target network updated at step 140000
Episode: 459, Reward: 16.0, Epsilon: 0.1002, Steps: 394
Episode: 460, Reward: 18.0, Epsilon: 0.0997, Steps: 384
Episode: 461, Reward: 16.0, Epsilon: 0.0992, Steps: 391
Target network updated at step 141000
Episode: 462, Reward: 15.0, Epsilon: 0.0987, Steps: 414
Episode: 463, Reward: 16.0, Epsilon: 0.0982, Steps: 439

Episode: 500, Reward: 18.0, Epsilon: 0.0816, Steps: 305
Model saved at episode 500
Episode: 501, Reward: 16.0, Epsilon: 0.0812, Steps: 369
Episode: 502, Reward: 20.0, Epsilon: 0.0808, Steps: 302
Target network updated at step 157000
Episode: 503, Reward: 15.0, Epsilon: 0.0804, Steps: 735
Episode: 504, Reward: 18.0, Epsilon: 0.0800, Steps: 276
Target network updated at step 158000
Episode: 505, Reward: 17.0, Epsilon: 0.0796, Steps: 713
Target network updated at step 159000
Episode: 506, Reward: 23.0, Epsilon: 0.0792, Steps: 793
Episode: 507, Reward: 13.0, Epsilon: 0.0788, Steps: 395
Episode: 508, Reward: 15.0, Epsilon: 0.0784, Steps: 318
Target network updated at step 160000
Episode: 509, Reward: 12.0, Epsilon: 0.0780, Steps: 312
Episode: 510, Reward: 18.0, Epsilon: 0.0776, Steps: 320
Episode: 511, Reward: 8.0, Epsilon: 0.0772, Steps: 306
Target network updated at step 161000
Episode: 512, Reward: 22.0, Epsilon: 0.0768, Steps: 705
Target network updated at step 162000
Episode: 513, Rewa